In [1]:
import serial
import csv

arduino_port = "COM5" #serial port of Arduino
baud = 115200 #arduino uno runs at 9600 baud
fileName="analog-data.csv" #name of the CSV file generated
ser = serial.Serial(arduino_port, baud)



In [ ]:
import csv
import serial
from time import sleep

arduino_port = "COM5"  # Replace with your actual port
fileName = "sensor_data.csv"

print("Connected to Arduino port: " + arduino_port)
file = open(fileName, "a")
print("Created file")

samples = 200
line = 0
sensor_data = []

# Column names for 5 sensors + label
column_names = ["Sensor1", "Sensor2", "Sensor3", "Sensor4", "Sensor5", "Label"]
sensor_data.append(column_names)

# Start serial connection
ser = serial.Serial(arduino_port, 115200)

# Countdown before starting
print("Starting in:")
for i in range(5, 0, -1):
    print(i)
    sleep(1)
print("Start collecting data...")

boo = True

while line < samples:
    getData = ser.readline().decode().strip()
    readings = getData.split(",")

    # Ensure correct number of readings
    if len(readings) != 5:
        continue  # skip invalid lines

    # Add label and notify user
    if line < samples / 2:
        label = "normal"
        if line == 0:
            print("Label now: normal")
    else:
        label = "กำหมัด"
        if line == samples // 2:
            print("Label now: กำหมัด")
            print("Please กำหมัด (make a fist) now!")
            sleep(2)  # Wait for user to act

    readings.append(label)
    sensor_data.append(readings)
    print(readings)

    line += 1

# Save to CSV
with open(fileName, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(sensor_data)

print("Data collection complete!")
file.close()


Connected to Arduino port: COM5
Created file
Starting in:
3
2
1
Start collecting data...
Label now: normal
['820', '826', '844', '882', '839', 'normal']
['820', '827', '844', '882', '838', 'normal']
['819', '827', '844', '883', '838', 'normal']
['814', '827', '844', '882', '837', 'normal']
['814', '827', '845', '882', '844', 'normal']
['812', '827', '845', '883', '848', 'normal']
['811', '826', '844', '883', '851', 'normal']
['812', '827', '845', '885', '858', 'normal']
['813', '827', '848', '887', '860', 'normal']
['815', '827', '848', '887', '860', 'normal']
['813', '827', '849', '887', '860', 'normal']
['814', '827', '848', '887', '860', 'normal']
['814', '827', '849', '887', '860', 'normal']
['815', '827', '849', '887', '860', 'normal']
['814', '826', '846', '885', '858', 'normal']
['813', '826', '846', '885', '857', 'normal']
['814', '826', '846', '885', '858', 'normal']
['814', '826', '846', '885', '858', 'normal']
['814', '826', '846', '885', '858', 'normal']
['814', '826', '846

KeyboardInterrupt: 